### The purpose of this notebook is to complete a data cleaning workflow from start to finish in order to validate the core functionality of our data cleaning python code

In [1]:
# imports

import pandas as pd
from core import *
# import gzip
# 
# from Bio.KEGG import REST
# from Bio.KEGG import Enzyme
# import re
# from Bio.KEGG import Compound
# import numpy as np

In [2]:
# create_kegg_df() - good to go for enzymes

enzyme_df = create_kegg_df('../datasets/KEGG_enzymes_all_data.gz', 'enzyme')

In [3]:
enzyme_df.shape

(7524, 16)

In [4]:
# create_kegg_df() - good to go for compounds

compound_df = create_kegg_df('../datasets/KEGG_compound_db_entries.gz', 'compound')

In [5]:
compound_df.shape

(18505, 8)

In [6]:
# select_promiscuous_enzymes - good to go

prom_df = select_promiscuous_enzymes(enzyme_df)

In [7]:
prom_df.shape

(549, 4)

In [8]:
# combine_substrates_products() - good to go

# prods = []
# subs = []
# p_s_combined = []

# for _, row in prom_df.iterrows():
#     prods.append(len(row['product']))
#     subs.append(len(row['substrate']))
    
combo_df = combine_substrates_products(prom_df)

# for _, row in combo_df.iterrows():
#      p_s_combined.append(len(row['product']))
        
# p_s_combined == [sum(x) for x in zip(prods, subs)]

In [9]:
combo_df.shape

(549, 2)

In [10]:
# explode_dataframe() - good to go
# parse_compound_ids() - good to go

exploded_df = explode_dataframe(combo_df, parse_compound_ids, 'product', ['entry'])

In [11]:
exploded_df.head()

,entry,product
0,1.1.1.1,C00071
1,1.1.1.1,C00004
2,1.1.1.1,C00080
3,1.1.1.1,C01450
4,1.1.1.1,C00226


In [12]:
# remove_cofactors() - 

cofactors_df = pd.read_csv('../datasets/cofactor_list.csv')
cofactors_df.head()

,Unnamed: 0,CPD,description
0,1,cpd:C00001,H2O; Water
1,2,cpd:C00002,ATP; Adenosine 5'-triphosphate
2,3,cpd:C00003,NAD+; NAD; Nicotinamide adenine dinucleotide; ...
3,4,cpd:C00004,NADH; DPNH; Reduced nicotinamide adenine dinuc...
4,5,cpd:C00005,NADPH; TPNH; Reduced nicotinamide adenine dinu...


In [13]:
# remove_cofactors() - good to go

clean_df = remove_cofactors(exploded_df, 'product', cofactors_df, 'CPD')

In [14]:
clean_df.head()

,entry,product
0,1.1.1.1,C00071
3,1.1.1.1,C01450
4,1.1.1.1,C00226
6,1.1.1.1,C01612
7,1.1.1.38,C00022


In [15]:
# create_negative_matches() - good to go

pos_df, neg_df = create_negative_matches(clean_df, 'entry', 'product')

In [16]:
pos_df.shape

(2144, 3)

In [17]:
neg_df.shape

(711105, 3)

In [18]:
# concatenate negative & positive data

master_df = pd.concat((pos_df, neg_df), axis=0)

In [19]:
master_df.shape

(713249, 3)

In [ ]:
# def join pubchem id onto 

# def join_pubchem_ids(master_df, pubchem_df, master_join_key, pubchem_join_key, pubchem_id_field):
    
#     pubchem_id_data = []

#     for _, row in pubchem_df.iterrows():
#         pubchem_id = parse_pubchem_ids(row[pubchem_id_field])
#         join_key = row[pubchem_join_key]
#         entry = {'pubchem_id': pubchem_id, pubchem_join_key: join_key}
#         pubchem_id_data.append(entry)

#     join_df = pd.DataFrame(pubchem_id_data)
    
#     master_df = master_df.merge(join_df, left_on=master_join_key, right_on=pubchem_join_key)
#     master_df = master_df.drop(columns=pubchem_join_key)
    
#     return master_df

In [20]:
master_df = join_pubchem_ids(master_df, compound_df, 'product', 'entry', 'dblinks')

In [24]:
master_df.shape

(713249, 4)

In [26]:
master_df = master_df.drop_duplicates()
master_df = master_df.dropna()
master_df.shape

(713249, 4)